# Person Detection using a MobileNetV2 Model

## Prerequisite
Download the dataset from [here](https://github.com/YatharthDedhia/Eklavya-Smart-Stand.git) and upload the Dataset to your drive. Mount your drive in your colab notebook.

In [ ]:
!pip install tensorflow_model_optimization


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import tensorflow as tf
import tensorflow.keras.layers as tfl
import tempfile
import sys
from PIL import Image
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Training and Testing sets 
We are splitting the dataset into a testing dataset and a validation dataset.

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
directory = "/content/drive/MyDrive/dataset"
train_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='training',
                                             seed=42)
test_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='validation',
                                             seed=42)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

## Data Augmentation
The images a rotated so the model is trained better.

In [ ]:
def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''
    ### START CODE HERE
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip("horizontal"))
    data_augmentation.add(RandomRotation(0.2))
    ### END CODE HERE
    
    return data_augmentation

In [ ]:
data_augmentation = data_augmenter()
data_augmentation = data_augmenter()
for image, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

## Preprocessing the dataset

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

## Instantiating the MobileNetV2 architecture

In [ ]:
IMG_SIZE = (224,224)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False,weights='imagenet')
base_model.summary()
type(base_model)

## Pruning the model

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = 6000
model_for_pruning = prune_low_magnitude(base_model)

# # `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

model_for_pruning.summary()

#Preparing the model
The layers of the model are freezed in transfer learning. Dropout is also applied to ignore certain neurons while training.

In [ ]:
def people_model(base_model,image_shape=IMG_SIZE, data_augmentation=data_augmenter()):
    ''' Define a tf.keras model for binary classification out of the MobileNetV2 model
    Arguments:
        image_shape -- Image width and height
        data_augmentation -- data augmentation function
    Returns:
    Returns:
        tf.keras.model
    '''
    
    
    input_shape = image_shape + (3,)
    
    # freeze the base model by making it non trainable
    base_model.trainable = False

    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape=input_shape) 
    
    # apply data augmentation to the inputs
    x = data_augmentation(inputs)
    
    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(x) 
    
    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x, training=False) 
    
    # add the new Binary classification layers
    # use global avg pooling to summarize the info in each channel
    x = tfl.GlobalAveragePooling2D()(x)  
    # include dropout with probability of 0.2 to avoid overfitting
    x = tfl.Dropout(0.2)(x)
        
    # use a prediction layer with one neuron (as a binary classifier only needs one)
    outputs = tfl.Dense(1)(x) 
    
    ### END CODE HERE
    
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [ ]:
model2 = people_model(base_model,IMG_SIZE, data_augmentation)

In [ ]:
base_learning_rate = 0.003
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model2.summary()

## Training the model

In [ ]:
initial_epochs = 10
history = model2.fit(train_dataset, validation_data=test_dataset, epochs=initial_epochs)

In [ ]:
acc = [0.] + history.history['accuracy']
val_acc = [0.] + history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Exporting the model
The original model is restored with the sparsed weights.

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model2)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, "/content/Saved_Model", include_optimizer=False)

## Applying Quantization
The model is first converted to a .tflite model. Then, dynamic range pruning method is used to reduce the size of the model.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_tflite_file = tempfile.mkstemp('.tflite')

with open("/content/Saved_Model.tflite", 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)
print("Size of tflite model ",sys.getsizeof(quantized_and_pruned_tflite_model))

## Converting the model to a C array
As we have to use our model on ESP32, converting it into a C array is essential .

In [ ]:
!apt-get update && apt-get -qq install xxd
!xxd -i "/content/Saved_Model.tflite" > model_data.cc